### Figure S6g

In [ ]:
library(dplyr)
library(ggplot2)
library(tidyr)
library(ComplexHeatmap)
library(circlize)

In [ ]:
sample_exp_df <- read.csv("./avg_expression/top_FC_average_expression_per_cluster_sample.csv",row.names=1)
col_order <- as.numeric(gsub("_.*", "", colnames(sample_exp_df)))
sample_exp_df_sorted <- sample_exp_df[, order(col_order)]
sample_exp_mtx <- as.matrix(sample_exp_df_sorted)


In [ ]:
sample_exp_mtx_zscored <- t(scale(t(sample_exp_mtx)))

group_colors <- c(
    '#ffd700', '#4d80e6', '#cccc4d', '#006374', '#1c9755',
    '#f5616f', '#afeeee', '#7ab8cc', '#cca3cc', '#2f7fc1',
    '#79b9f2', '#0d5886', '#082567', '#dcdcdc', '#d3d3d3',
    '#7dcc7d', '#00bfff', '#87ceeb', '#fa8072', '#48d1cc',
    '#469594'
)

In [ ]:
group <- as.numeric(gsub("_.*", "", colnames(sample_exp_mtx)))
names(group_colors) <- unique(group)
col_anno <- HeatmapAnnotation(Group = as.factor(group),
                              col = list(Group = group_colors))

p <- Heatmap(sample_exp_mtx_zscored,
        top_annotation = col_anno,
        cluster_columns = FALSE,
        cluster_rows = FALSE,
        show_row_names = TRUE,
        show_column_names = FALSE
)

print(p)

### FigS7b

In [ ]:
library(dplyr)
library(EnhancedVolcano)

In [ ]:
results <- read.csv("./DEG_in_niche12.csv")
results$scores <- abs(results$scores)

cut_off_score <- 50
cut_off_logFC <- 0.8

results$Sig = ifelse(results$scores > cut_off_score & 
                abs(results$logfoldchanges) >= cut_off_logFC, 
                ifelse(results$logfoldchanges> cut_off_logFC ,'Up','Down'),'None')

In [ ]:
label_genes <- c(
                "KRT5","MUC1","CYC1","GPX3","CD24","CD47","WNT7A","MDK","S100A6","S100A10","PRKCI","LAMC1","EIF3",
                "TMSB10","CRABP2","LAPTM4B","HOXB1","CDH16","HAND1","IGFBP6","SLC3A1",
                "IL5RA","ADAMTS18","CDC20B","CNMD","FGL2","SPARC","CAV1","CAP5","TPPP3","PIFO","OVGP1","CROCC2"
                )

options(repr.plot.width = 13, repr.plot.height = 7)

p <- ggplot(results, aes(x = logfoldchanges, y = scores, colour=Sig)) +
  geom_point(alpha=0.8, size=2) +
  scale_color_manual(values=c("#546de5", "#d2dae2","#ff4757"))+
  geom_vline(xintercept=c(-cut_off_logFC,cut_off_logFC),lty=4,col="black",lwd=0.4) +
  geom_hline(yintercept = cut_off_score,
             lty=4,col="black",lwd=0.4) +
  xlim(-4.5, 3.5) +
  ylim(0, 500) +
  labs(x="log2(Fold Change)",
       y="T score")+
  theme_bw()+
  ggtitle("")+
  theme(plot.title = element_text(hjust = 0.5), 
        legend.position="right", 
        legend.title = element_blank()
  ) +
  geom_text_repel(
    aes(label = ifelse(names %in% label_genes, as.character(names), "")),
    size = 4, 
    color = "black",
    fontface = "bold",
    box.padding = 0,
    point.padding = 0,
    segment.size = 0.5,
    segment.color = "gray50",
    segment.linetype = "solid",
    segment.alpha = 0.8,
    min.segment.length = 0,
    max.overlaps = Inf,
    force = 50,
    force_pull = 5,
    nudge_x = 0.2,
    nudge_y = 0.2
  )
